In [1]:
import pandas as pd
import config
import os
import utils as ut
import stf

# Search
handled using pickle files which load faster than the STF or Excel files and allow basic searches

In [2]:
def load_pickle_files(directory):
    dfs = []  # Initialize an empty list to store dataframes

    # Traverse through all pickle files in the given directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pkl'):  # Check if the file is a pickle file
                filepath = os.path.join(root, file)
                df = pd.read_pickle(filepath)  # Load the pickle file into a DataFrame
                df['full_filepath'] = filepath.replace(config.PICKLE,'')[1:]  # Add a new column with the full file path
                dfs.append(df)  # Append the DataFrame to the list

    master_df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
    
    return master_df  # Return the final DataFrame

sdf = load_pickle_files(config.PICKLE)


### Perform a basic search
sdf can be Value or ID, those are the two columns

In [3]:
search_value = "recieve"
results = sdf[sdf['Value'].str.contains(search_value, case=False, na=False)]

results.head(25)

,ID,Value,full_filepath
123049,s_17,Recieve Quest,conversation\first_sister_fight_controller.pkl
140790,s_17,Recieve Quest,conversation\rancor_boss_fight_controller.pkl
211866,task06_journal_entry_title,Recieve the Datapad,quest\ground\legacy_button_jawatracks_pt2.pkl
211878,task11_journal_entry_description,Recieved Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2.pkl
211879,task11_journal_entry_title,Recieve Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2.pkl
211892,task01_journal_entry_description,Recieved Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2a.pkl
211893,task01_journal_entry_title,Recieve Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2a.pkl
211919,task07_journal_entry_title,Recieve the Datapad,quest\ground\legacy_button_jawatracks_pt2b.pkl
211926,task12_journal_entry_description,Recieved Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2b.pkl
211927,task12_journal_entry_title,Recieve Zef's Datapad,quest\ground\legacy_button_jawatracks_pt2b.pkl


In [ ]:
st1 = results.iloc[0,1]
exf = ut.get_excel(results.iloc[0,2])

In [ ]:
st2 = st1.replace('automatic the', 'automate the')
print(st2)

In [ ]:
df = pd.read_excel(exf)
df.head()

In [ ]:
for index, row in df.iterrows():
    if row['ID'] == "loot_schem_slicingstation":
        print(row['Value'])

### Find and Replace
This operation seems best to do this:
- create a dattaframe of all the ID/Value pairs with the file they are from
- go through and read the STF file and replace the value with the new value
- then create the excel file and publish it


In [ ]:
search_value = "automatic the"
results = sdf[sdf['Value'].str.contains(search_value, case=False, na=False)]

results.head(25)

In [ ]:
data = []
data.append(["Value", "Recieve", "Receive"])
# now perform a search for this
for d in data:
    print(d)
    results = sdf[sdf[d[0]].str.contains(d[1], case=True, na=False)]
    for i in range(len(results)):
        exf = ut.get_excel(results.iloc[i,2])
        print(exf)
        prdf = pd.read_excel(exf)
        ndata = []
        for j in range(len(prdf)):
            if prdf.iloc[j,0] == results.iloc[i,0]:
                ndata.append([prdf.iloc[j,0], prdf.iloc[j,1].replace(d[1],d[2])])
            else:
                ndata.append([prdf.iloc[j,0], prdf.iloc[j,1]])
        pndf = pd.DataFrame(ndata, columns=['ID', 'Value'])
        pndf.to_excel(exf, index=False)
        nexc = exf.replace(config.EXCEL,'')[1:]
        print(nexc.replace('.xlsx', ''))
        stf.excel_publish(nexc.replace('.xlsx', ''))